In [1]:
import os
import os.path
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt
import cv2

folderPath ='/mnt/DeepLearning/hand/hand143_panopticdb/' #Put your local path here
jsonPath = folderPath +'hands_v143_14817.json'

plt.rcParams['figure.figsize'] = (20,20)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

with open(jsonPath, 'r') as fid:
    dat_all = json.load(fid)
    data_all= dat_all['root']

In [2]:
print(len(data_all)) 
data_train = data_all[:13000]
data_valid = data_all[13000:14000]
data_test = data_all[14000:]

14817


In [3]:
import shutil, os
path = "/mnt/DeepLearning/hand/hand143_panopticdb/"

train_files = [file['img_paths'] for file in data_train] 
valid_files = [file['img_paths'] for file in data_valid] 
test_files = [file['img_paths'] for file in data_test]

for file in train_files:
    file_image = file.replace("imgs/","")
    shutil.copy(path +file, path + "train2017/" + file_image)

for file in valid_files:
    file_image = file.replace("imgs/","")
    shutil.copy(path +file, path + "val2017/" + file_image)

for file in test_files:
    file_image = file.replace("imgs/","")
    shutil.copy(path +file, path + "test2017/" + file_image)

# Training 

In [4]:
dataset = {}
dataset['info'] = []
dataset["licenses"] = []
dataset['categories'] = []
dataset["images"] = []
dataset['annotations'] = []


# Info 
info = {}
info["contributor"] = "Fruit Lab"
info["date_created"] = '4/5/2020' 
info["description"] = "Hand Dataset Training" 
info["url"] = "https://github.com/Fruit-Lab"
info["version"] = "7GB"
info["year"] = "2020"
dataset['info'].append(info)


# License 
license = {} 
license['id'] = 1 
license['name'] = "Fruit Lab"
license['url'] = "https://github.com/Fruit-Lab"
dataset["licenses"].append(license)


# Categories 
category = {}
category['id'] = 1 
category['name'] = "hand"
category["supercategory"] = "body"
dataset['categories'].append(category)

In [5]:
for i,data in enumerate(data_train):
    # --------- Images ------------- 
    data_temp = {}
    data_temp["id"] = i 
    data_temp["width"] = data['img_width'] 
    data_temp["height"] = data['img_height']
    
    file_name = data['img_paths'].replace("imgs/","")
    data_temp["file_name"] = file_name
    
    data_temp["license"] = 1 
    data_temp["flickr_url"] = ""
    data_temp["coco_url"] = ""
    data_temp["date_captured"] = "15/5/2020"
    dataset["images"].append(data_temp)
    
    # ---------- Annotations --------
    tmp = {}
    x_list = [item[0] for item in data['joint_self']]
    y_list = [item[1] for item in data['joint_self']]
    
    x_min = min(x_list) - 25
    y_min = min(y_list) - 15
    
    x_max = max(x_list) + 15
    y_max = max(y_list) + 20
    
    width = x_max - x_min 
    height = y_max - y_min
    
    tmp['id'] = i 
    tmp["image_id"] = i
    tmp["category_id"] = 1
    tmp["segmentation"] = [[]]
    tmp["area"] = width *  height
    tmp["bbox"] = [x_min,y_min,width,height] 
    tmp["iscrowd"] = 0 # No more than 2 people
    dataset['annotations'].append(tmp)
    
    
    # ------------------ Big Size 512 ------------------
    
    # Tạo ra ROI có tay ở trung tâm ( Shape = 512x512) 
    xmin_ROI = data['objpos'][0] - 256
    ymin_ROI = data['objpos'][1] - 256
    
    xmax_ROI = data['objpos'][0] + 256
    ymax_ROI = data['objpos'][1] + 256
    
    # Read and cut ROI 
    imgPath = "/mnt/DeepLearning/hand/hand143_panopticdb/train2017/" 
    img = cv2.imread(imgPath+ file_name)
    roi = img[int(ymin_ROI):int(ymax_ROI), int(xmin_ROI):int(xmax_ROI),:]
    file_name2 = file_name.replace(".jpg","") + "_512.jpg" 
    print(file_name2)
    
    if os.path.exists(imgPath+file_name2) == False: # Chinh #
        cv2.imwrite(imgPath+file_name2 , roi)
    
    
    # -----Thêm info của ảnh 512x512----
    data_temp2 = {}
    count = len(data_train) + i  # Chinh #
    data_temp2["id"] = count 
    
    data_temp2["width"] = xmax_ROI - xmin_ROI
    data_temp2["height"] = ymax_ROI  - ymin_ROI
    
    data_temp2["file_name"] = file_name2
    
    data_temp2["license"] = 1 
    data_temp2["flickr_url"] = ""
    data_temp2["coco_url"] = ""
    data_temp2["date_captured"] = "15/5/2020"
    
    dataset["images"].append(data_temp2)
    
    
    # --- Annotation of 512x512 ---
    tmp2 = {}
    # X
    xmin_ROI_bbox = x_min - xmin_ROI 
    ymin_ROI_bbox = y_min - ymin_ROI
    
    # Y 
    xmax_ROI_bbox = 512 - (xmax_ROI - x_max)
    ymax_ROI_bbox = 512 - (ymax_ROI - y_max)
    
    # W and H 
    width2 = xmax_ROI_bbox  - xmin_ROI_bbox
    height2 = ymax_ROI_bbox - ymin_ROI_bbox
    
    tmp2['id'] = count
    tmp2["image_id"] = count
    tmp2["category_id"] = 1
    tmp2["segmentation"] = [[]]
    tmp2["area"] = width2 *  height2
    tmp2["bbox"] = [ xmin_ROI_bbox,ymin_ROI_bbox,width2,height2] 
    tmp2["iscrowd"] = 0 # No more than 2 people
    dataset['annotations'].append(tmp2)
    
    
import json
with open("./Annotations/CMU2020_train.json", "w") as outfile: 
    json.dump(dataset, outfile)

00000000_512.jpg
00000001_512.jpg
00000002_512.jpg
00000003_512.jpg
00000004_512.jpg
00000005_512.jpg
00000006_512.jpg
00000007_512.jpg
00000008_512.jpg
00000009_512.jpg
00000010_512.jpg
00000011_512.jpg
00000012_512.jpg
00000013_512.jpg
00000014_512.jpg
00000015_512.jpg
00000016_512.jpg
00000017_512.jpg
00000018_512.jpg
00000019_512.jpg
00000020_512.jpg
00000021_512.jpg
00000022_512.jpg
00000023_512.jpg
00000024_512.jpg
00000025_512.jpg
00000026_512.jpg
00000027_512.jpg
00000028_512.jpg
00000029_512.jpg
00000030_512.jpg
00000031_512.jpg
00000032_512.jpg
00000033_512.jpg
00000034_512.jpg
00000035_512.jpg
00000036_512.jpg
00000037_512.jpg
00000038_512.jpg
00000039_512.jpg
00000040_512.jpg
00000041_512.jpg
00000042_512.jpg
00000043_512.jpg
00000044_512.jpg
00000045_512.jpg
00000046_512.jpg
00000047_512.jpg
00000048_512.jpg
00000049_512.jpg
00000050_512.jpg
00000051_512.jpg
00000052_512.jpg
00000053_512.jpg
00000054_512.jpg
00000055_512.jpg
00000056_512.jpg
00000057_512.jpg
00000058_512.j

error: OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [10]:
print(data['objpos'][0])
print(data['objpos'][1])

import math 
dist = math.hypot(data['objpos'][0] - 0, data['objpos'][1] - 256)
dist

923.584
242.628


923.6807973753703